In [1]:
import transformers
from transformers import pipeline

pipe = pipeline(task='text-classification', model='ProsusAI/finbert')

In [2]:
total_score = 0
num_articles = 0

In [3]:
import pandas as pd

file_path = fr'C:\Users\ivane\Desktop\Dissertation-2\data\maltabusinessweeklyArticles.csv'

df = pd.read_csv(fr'{file_path}')

df.head()

,headline,url
0,"Inflation, environment main concerns for Malte...",https://maltabusinessweekly.com/inflation-envi...
1,16 local councils in financial trouble as they...,https://maltabusinessweekly.com/16-local-counc...
2,Parliament approves 2025 Budget estimates for ...,https://maltabusinessweekly.com/parliament-app...
3,PwC Malta’s CEO Confidence Tracker reveals gen...,https://maltabusinessweekly.com/pwc-maltas-ceo...
4,Court revokes permit to add six floors on Rame...,https://maltabusinessweekly.com/court-revokes-...


In [9]:
for index, row in df.iterrows():
    print(f'Headline: {row["headline"]}')
    print(f'URL: {row["url"]}')
    
    sentiment = pipe(row['headline'])[0]
    print(f'Sentiment: {sentiment}')
    
    print('-' * 40)
    if sentiment['label'] == 'positive':
        total_score += sentiment['score']
        num_articles += 1
    elif sentiment['label'] == 'negative':
        total_score -= sentiment['score']
        num_articles += 1

final_score = total_score / num_articles if num_articles > 0 else 0

Headline: Inflation, environment main concerns for Maltese citizens – Eurobarometer
URL: https://maltabusinessweekly.com/inflation-environment-main-concerns-for-maltese-citizens-eurobarometer/27929/
Sentiment: {'label': 'neutral', 'score': 0.714597761631012}
----------------------------------------
Headline: 16 local councils in financial trouble as they owe more money than they have
URL: https://maltabusinessweekly.com/16-local-councils-in-financial-trouble-as-they-owe-more-money-than-they-have/27924/
Sentiment: {'label': 'negative', 'score': 0.9649041891098022}
----------------------------------------
Headline: Parliament approves 2025 Budget estimates for all government ministries
URL: https://maltabusinessweekly.com/parliament-approves-2025-budget-estimates-for-all-government-ministries/27915/
Sentiment: {'label': 'neutral', 'score': 0.8978353142738342}
----------------------------------------
Headline: PwC Malta’s CEO Confidence Tracker reveals general lack of enthusiasm around 20